In [ ]:
from census import Census
from config import (census_key)
from us import states
import pandas as pd


c = Census(census_key, year=2013)

In [ ]:
#c.sf1.state_county_tract('NAME', states.TX.fips, '113', Census.ALL)

In [ ]:
# Run Census Search to retrieve data on all zip codes (2013 ACS5 Census)
# See: https://github.com/CommerceDataService/census-wrapper for library documentation
# See: https://gist.github.com/afhaque/60558290d6efd892351c4b64e5c01e9b for labels
census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
                          "B19301_001E",
                          "B17001_002E", "B23025_005E"), {'for': 'zip code tabulation area:*'})

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "B23025_005E": "Unemployment Count",
                                      "NAME": "Name", "zip code tabulation area": "Zipcode"})

# Add in Poverty Rate (Poverty Count / Population)
census_pd["Poverty Rate"] = 100 * \
    census_pd["Poverty Count"].astype(
        int) / census_pd["Population"].astype(int)

# Final DataFrame
census_pd = census_pd[["Zipcode", "Population", "Median Age", "Household Income",
                       "Per Capita Income", "Poverty Count", "Poverty Rate", "Unemployment Count"]]

# Visualize
print(len(census_pd))
census_pd.head()

In [ ]:
dallasZips = pd.read_csv("../Resources/DallasZipCodes.csv")
dallasZips.head()

In [ ]:
print(census_pd.dtypes)
print(dallasZips.dtypes)

In [ ]:
census_pd["Zipcode"] = census_pd["Zipcode"].astype(int)
mask = [zipcode in dallasZips.ZipCode.values for zipcode in census_pd.Zipcode]
dallasSub = census_pd[mask]

In [ ]:
dallasSub.head()

In [ ]:
dallasSubMerge = pd.merge(dallasSub,dallasZips[['ZipCode','City']],left_on='Zipcode', right_on='ZipCode', how='inner')
dallasSubMerge = dallasSubMerge.drop(columns=["ZipCode"])
dallasSubMerge

In [ ]:
#drop NaN values
DroppedData = dallasSubMerge.dropna(how="any")


DroppedData.describe()

In [ ]:
dallasSubMerge.to_csv("../Resources/DallasCensusData.csv", index=False)